##1. Bibliotecas necessárias

In [ ]:
import pip
import pandas as pd

In [ ]:
!pip install dask[complete]

     |████████████████████████████████| 830 kB 2.4 MB/s 
     |████████████████████████████████| 133 kB 51.2 MB/s 
     |████████████████████████████████| 822 kB 49.8 MB/s 
     |████████████████████████████████| 802 kB 44.7 MB/s 
     |████████████████████████████████| 802 kB 36.2 MB/s 
     |████████████████████████████████| 793 kB 51.7 MB/s 
     |████████████████████████████████| 793 kB 39.5 MB/s 
     |████████████████████████████████| 791 kB 37.8 MB/s 
     |████████████████████████████████| 786 kB 37.9 MB/s 
     |████████████████████████████████| 779 kB 40.0 MB/s 
     |████████████████████████████████| 778 kB 42.7 MB/s 
     |████████████████████████████████| 776 kB 36.3 MB/s 
     |████████████████████████████████| 769 kB 40.1 MB/s 
     |████████████████████████████████| 766 kB 44.8 MB/s 
     |████████████████████████████████| 1.0 MB 39.7 MB/s 
     |████████████████████████████████| 722 kB 43.1 MB/s 
     |████████████████████████████████| 722 kB 34.9 MB/s 
     |█████████

In [ ]:
#importando o dask 
import dask.array as da
import dask.dataframe as dd

Cria dask array de dimensoes de 10000 por 10000

##2. Importando base de dados para analise


In [ ]:
#acessando drive
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##lendo a base de dados que esta quase completa
df = dd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/df2_completo.csv')

In [ ]:
#verificando as infos
df = df.compute()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119143 entries, 0 to 674
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119143 non-null  object 
 1   order_item_id                  118310 non-null  float64
 2   product_id                     118310 non-null  object 
 3   seller_id                      118310 non-null  object 
 4   shipping_limit_date            118310 non-null  object 
 5   price                          118310 non-null  float64
 6   freight_value                  118310 non-null  float64
 7   payment_sequential             119140 non-null  float64
 8   payment_type                   119140 non-null  object 
 9   payment_installments           119140 non-null  float64
 10  payment_value                  119140 non-null  float64
 11  review_id                      118146 non-null  object 
 12  review_score                   11

##3. Trabalhando a base de dados geolocation

In [ ]:
#importando a base com geolocalizações
df_geolocation = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_geolocation_dataset.csv")

In [ ]:
df_geolocation.head(10)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
5,1012,-23.547762,-46.635361,são paulo,SP
6,1047,-23.546273,-46.641225,sao paulo,SP
7,1013,-23.546923,-46.634264,sao paulo,SP
8,1029,-23.543769,-46.634278,sao paulo,SP
9,1011,-23.547640,-46.636032,sao paulo,SP


para um mesmo codigo postal tem-se lat e long diferentes, como por exemplo ali no 1046
vamos fazer entao a media para um mesmo codigo postal

###3.1 Verificando dados faltantes

####3.1.1 Teste com pandas 

In [ ]:
#teste
import pandas as pd
teste_geo = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/IC/eccomerce/database/dataframes/olist_geolocation_dataset.csv")

In [ ]:
teste_geo.value_counts()

geolocation_zip_code_prefix  geolocation_lat  geolocation_lng  geolocation_city  geolocation_state
88220                        -27.102099       -48.629613       itapema           SC                   314
6414                         -23.495901       -46.874687       barueri           SP                   189
                             -23.490618       -46.869004       barueri           SP                   127
5145                         -23.506049       -46.717377       sao paulo         SP                   126
22620                        -23.005514       -43.375964       rio de janeiro    RJ                   102
                                                                                                     ... 
39400                        -16.729617       -43.869854       montes claros     MG                     1
                             -16.729663       -43.865562       montes claros     MG                     1
                             -16.729720       -43.864

####3.1.2 Dados nulos


In [ ]:
#utilizando o drop_duplicates, retorna um novo dataframe sem as duplicações
new_df_geo = df_geolocation.drop_duplicates()

In [ ]:
new_df_geo.head(15)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
5,1012,-23.547762,-46.635361,são paulo,SP
6,1047,-23.546273,-46.641225,sao paulo,SP
7,1013,-23.546923,-46.634264,sao paulo,SP
8,1029,-23.543769,-46.634278,sao paulo,SP
9,1011,-23.547640,-46.636032,sao paulo,SP


Agora, faz-se a media da latitude e longitude para o mesmo cep

In [ ]:
#utilizando groupby
new_df_geo = new_df_geo.groupby(by="geolocation_zip_code_prefix").mean()


In [ ]:
new_df_geo.head(15)

,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.550227,-46.634039
1002,-23.547657,-46.634991
1003,-23.549000,-46.635582
1004,-23.549829,-46.634792
1005,-23.549547,-46.636406
1006,-23.550127,-46.636044
1007,-23.549962,-46.637204
1008,-23.546000,-46.635877
1009,-23.546891,-46.636498


Para cada vendedor e para cada cliente ter um cep associado

In [ ]:
#Identificando o cliente
#left para o df completo e rigth para o com as localizações
costumer = df.merge(new_df_geo, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix')

In [ ]:
#certo, cada cliente tem um lat e um long
costumer.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,geolocation_lat,geolocation_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1.0,credit_card,2.0,72.19,97ca439bc427b48bc1cd7177abe71365,5.0,0,"Perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,27277.0,volta redonda,SP,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265
1,bd31b009e1dbc47fc7c250b1e2cf5440,1.0,92e2d2146e433cd4d1f09a3f8633ead0,4a3ca9315b744ce9f8e9374361493884,2018-05-02 03:51:18,52.90,22.95,1.0,boleto,1.0,75.85,1a99824f74d0922e36ad6631e7e44288,3.0,0,0,2018-05-09 00:00:00,2018-05-10 02:04:02,2c23918ee23c1406b9cb5f78d4b2062a,delivered,2018-04-24 12:07:51,2018-04-25 03:51:18,2018-04-25 15:33:00,2018-05-08 22:24:49,2018-05-21 00:00:00,8d500d527a1d67efec12faf28b61885b,28013,campos dos goytacazes,RJ,14940.0,ibitinga,SP,cama_mesa_banho,53.0,714.0,1.0,1200.0,33.0,18.0,26.0,-21.763186,-41.310265
2,bb4eb0196897c20281a61f75ce23211c,1.0,8cee1d824765335c48ccc515c2ecf4b8,4a3ca9315b744ce9f8e9374361493884,2017-07-28 14:50:11,96.00,15.42,1.0,credit_card,9.0,111.42,2e6dd84a89e6ca047502168d21c14c85,5.0,0,"O tapete e lindo, é muito grande! Da um charme...",2017-08-04 00:00:00,2017-08-04 23:33:50,c7d65abc07da65a413738fc854d8112a,delivered,2017-07-24 14:36:16,2017-07-24 14:50:11,2017-07-26 20:22:05,2017-08-03 17:37:42,2017-08-15 00:00:00,81fde78227bbac716944e215b6895639,28013,campos dos goytacazes,RJ,14940.0,ibitinga,SP,cama_mesa_banho,54.0,385.0,2.0,900.0,30.0,9.0,20.0,-21.763186,-41.310265
3,f01cc1e4fe1833e42f696b0dcb0d77ef,1.0,281886d5650643134700ec611ac33532,218d46b86c1881d022bce9c68a7d4b15,2017-11-20 19:55:23,220.00,18.79,1.0,credit_card,3.0,238.79,822d19988a38c9693b58d2e44cf75c47,3.0,0,0,2017-11-28 00:00:00,2017-11-28 23:54:52,09fa775305489c7a8c4782738c7e67be,delivered,2017-11-10 19:45:33,2017-11-10 19:55:23,2017-11-14 20:51:55,2017-11-27 21:24:09,2017-12-06 00:00:00,93b503b0333e115f56d38411df95d833,28013,campos dos goytacazes,RJ,14070.0,ribeirao preto,SP,esporte_lazer,31.0,1327.0,1.0,1100.0,16.0,16.0,16.0,-21.763186,-41.310265
4,f4776a61a23801c349ab69bd0fd774e2,1.0,5a6e53c3b4e8684b13388d6aa4afdf12,7299e27ed73d2ad986de7f7c77d919fa,2017-05-23 04:02:17,16.99,15.10,1.0,boleto,1.0,32.09,b4f3ebc22714dc945c085a9f2e7a4192,5.0,0,0,2017-05-26 00:00:00,2017-05-26 23:31:23,db06dace4f0966b844c9d8ef3caed3f1,delivered,2017-05-16 14:29:36,2017-05-17 04:02:17,2017-05-17 10:40:42,2017-05-26 00:03:43,2017-06-07 00:00:00,36bf4779c5d09a3ce8162eb5acdeb711,28013,campos dos goytacazes,RJ,38440.0,araguari,MG,utilidades_domesticas,45.0,176.0,2.0,200.0,19.0,9.0,12.0,-21.763186,-41.310265


Problema: agora tem-se duas colunas com as informações de geolocalizacao
retira-se as erradas: costumer_lat e costumer_long

In [ ]:
#a coluna para latitude do cliente recebe o valor correto
costumer['costumer_lat'] = costumer['geolocation_lat']

In [ ]:
#apaga a variavel geolocation que nao precisa mais
costumer = costumer.drop(['geolocation_lat'],axis=1)

In [ ]:
#a coluna para longitude do cliente recebe o valor correto
costumer['costumer_lng'] = costumer['geolocation_lng']

In [ ]:
#apaga a variavel geolocation que nao precisa mais
costumer = costumer.drop(['geolocation_lng'],axis=1)

In [ ]:
costumer.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,costumer_lat,costumer_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1.0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1.0,credit_card,2.0,72.19,97ca439bc427b48bc1cd7177abe71365,5.0,0,"Perfeito, produto entregue antes do combinado.",2017-09-21 00:00:00,2017-09-22 10:57:03,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,27277.0,volta redonda,SP,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,-21.763186,-41.310265
1,bd31b009e1dbc47fc7c250b1e2cf5440,1.0,92e2d2146e433cd4d1f09a3f8633ead0,4a3ca9315b744ce9f8e9374361493884,2018-05-02 03:51:18,52.90,22.95,1.0,boleto,1.0,75.85,1a99824f74d0922e36ad6631e7e44288,3.0,0,0,2018-05-09 00:00:00,2018-05-10 02:04:02,2c23918ee23c1406b9cb5f78d4b2062a,delivered,2018-04-24 12:07:51,2018-04-25 03:51:18,2018-04-25 15:33:00,2018-05-08 22:24:49,2018-05-21 00:00:00,8d500d527a1d67efec12faf28b61885b,28013,campos dos goytacazes,RJ,14940.0,ibitinga,SP,cama_mesa_banho,53.0,714.0,1.0,1200.0,33.0,18.0,26.0,-21.763186,-41.310265
2,bb4eb0196897c20281a61f75ce23211c,1.0,8cee1d824765335c48ccc515c2ecf4b8,4a3ca9315b744ce9f8e9374361493884,2017-07-28 14:50:11,96.00,15.42,1.0,credit_card,9.0,111.42,2e6dd84a89e6ca047502168d21c14c85,5.0,0,"O tapete e lindo, é muito grande! Da um charme...",2017-08-04 00:00:00,2017-08-04 23:33:50,c7d65abc07da65a413738fc854d8112a,delivered,2017-07-24 14:36:16,2017-07-24 14:50:11,2017-07-26 20:22:05,2017-08-03 17:37:42,2017-08-15 00:00:00,81fde78227bbac716944e215b6895639,28013,campos dos goytacazes,RJ,14940.0,ibitinga,SP,cama_mesa_banho,54.0,385.0,2.0,900.0,30.0,9.0,20.0,-21.763186,-41.310265
3,f01cc1e4fe1833e42f696b0dcb0d77ef,1.0,281886d5650643134700ec611ac33532,218d46b86c1881d022bce9c68a7d4b15,2017-11-20 19:55:23,220.00,18.79,1.0,credit_card,3.0,238.79,822d19988a38c9693b58d2e44cf75c47,3.0,0,0,2017-11-28 00:00:00,2017-11-28 23:54:52,09fa775305489c7a8c4782738c7e67be,delivered,2017-11-10 19:45:33,2017-11-10 19:55:23,2017-11-14 20:51:55,2017-11-27 21:24:09,2017-12-06 00:00:00,93b503b0333e115f56d38411df95d833,28013,campos dos goytacazes,RJ,14070.0,ribeirao preto,SP,esporte_lazer,31.0,1327.0,1.0,1100.0,16.0,16.0,16.0,-21.763186,-41.310265
4,f4776a61a23801c349ab69bd0fd774e2,1.0,5a6e53c3b4e8684b13388d6aa4afdf12,7299e27ed73d2ad986de7f7c77d919fa,2017-05-23 04:02:17,16.99,15.10,1.0,boleto,1.0,32.09,b4f3ebc22714dc945c085a9f2e7a4192,5.0,0,0,2017-05-26 00:00:00,2017-05-26 23:31:23,db06dace4f0966b844c9d8ef3caed3f1,delivered,2017-05-16 14:29:36,2017-05-17 04:02:17,2017-05-17 10:40:42,2017-05-26 00:03:43,2017-06-07 00:00:00,36bf4779c5d09a3ce8162eb5acdeb711,28013,campos dos goytacazes,RJ,38440.0,araguari,MG,utilidades_domesticas,45.0,176.0,2.0,200.0,19.0,9.0,12.0,-21.763186,-41.310265


Para o vendedor

In [ ]:
seller = costumer.merge(new_df_geo, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix')

In [ ]:
#apaga a variavel geolocation que nao precisa mais
#a coluna para latitude do vendedor recebe o valor correto
seller['seller_lat'] = seller['geolocation_lat']
seller = seller.drop(['geolocation_lat'],axis=1)

In [ ]:
#apaga a variavel geolocation que nao precisa mais
#a coluna para long do vendedor recebe o valor correto
seller['seller_lng'] = seller['geolocation_lng']
seller = seller.drop(['geolocation_lng'],axis=1)

In [ ]:
seller.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value', 'review_id',
       'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'costumer_lat', 'costumer_lng', 'seller_lat', 'seller_lng'],
      dtype='object')

In [ ]:
costumer.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value', 'review_id',
       'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'seller_zip_code_prefix', 'seller_city', 'seller_state',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'costumer_lat', 'costumer_lng'],
      dtype='object')

In [ ]:
#seller = seller.compute()

In [ ]:
#salvando df geolocations prontinho
from google.colab import files
seller.to_csv('olist_geolocated_transactions.csv', index = False)
files.download('olist_geolocated_transactions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##4. Verificando se funcionou plotando graficos 

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(seller, lat = 'seller_lat', lon = 'seller_lng',
                        color_discrete_sequence=["fuchsia"], zoom=3, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0
fig.show()

In [ ]:
import plotly.express as px
fig = px.density_mapbox(seller, lat='seller_lat', lon='seller_lng', z='price', radius=10,
                        center=dict(lat=0, lon=180), zoom=500,
                        mapbox_style="stamen-terrain")
fig.show()